In [ ]:
import pandas as pd
import os

In [ ]:
train = pd.read_csv(os.path.join('prepared', 'train.csv'))
test = pd.read_csv(os.path.join('prepared', 'test.csv'))

train.drop(columns=['GT_NO2'], inplace=True)

In [ ]:
train['is_train'] = 1
test['is_train'] = 0
whole = pd.concat([train, test], axis=0, ignore_index=True).reset_index(drop=True)

whole.info()

In [ ]:
whole['is_train'].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split


X, y = whole.drop(columns=['is_train'], axis=1), whole['is_train']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4)
X_train = pd.DataFrame(X_train, columns=X.columns)
X_test = pd.DataFrame(X_test, columns=X.columns)

In [ ]:
import lightgbm as lgb
import optuna
from sklearn.model_selection import cross_val_score, GroupKFold


def define_model(trial):
    params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'max_bin': trial.suggest_int('max_bin', 10, 20),
        'num_leaves': trial.suggest_int('num_leaves', 10, 50),
        'max_depth': trial.suggest_int('max_depth', 5, 8),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 200, 500),
        'tree_learner': 'voting',
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0, log=True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 0.8, log=True),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 50, 250),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-2, 1e-1, log=True),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 5),
        'device': 'cpu',
        'n_jobs': -1,
        'random_state': 4,
        'verbosity': -1,
        'is_unbalance': True,
    }
    return lgb.LGBMClassifier(**params)

def objective(trial):
    model = define_model(trial)
    gkf = GroupKFold(n_splits=X['Season'].nunique())
    scores = cross_val_score(model, X, y, groups=X['Season'], cv=gkf, n_jobs=-1, scoring='accuracy')
    return scores.mean()

In [ ]:
study_lightgbm = optuna.create_study(direction='maximize', study_name='GeoAIWithLightGBM', sampler=optuna.samplers.TPESampler())
study_lightgbm.optimize(objective, n_trials=50)

In [ ]:
lightgbm = define_model(study_lightgbm.best_trial)
lightgbm.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report


print(classification_report(y_test, lightgbm.predict(X_test)))

In [ ]:
lgb.plot_importance(lightgbm, figsize=(20, 12), dpi=200)

In [ ]:
lgb.plot_tree(lightgbm, precision=2, figsize=(20, 12), show_info=['data_percentage'], dpi=200, orientation='vertical')

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay


ConfusionMatrixDisplay.from_predictions(y_true=y_test, y_pred=lightgbm.predict(X_test), cmap='Greys')

In [ ]:
from sklearn.model_selection import LearningCurveDisplay


LearningCurveDisplay.from_estimator(lightgbm, X, y, cv=GroupKFold(n_splits=X['Season'].nunique()), groups=X['Season'], n_jobs=-1, random_state=4, scoring='accuracy')